In [1]:
import pandas as pd
import numpy as np
import hdf5_getters as hd
import glob
import sklearn
from sklearn import preprocessing

In [2]:
#To see if all the 10,000 files are available for data extraction
h5_dir = "MillionSongSubset" #<file_path where the song_dataset is stored>

# Get a list of all .h5 files in the directory and its subdirectories
h5_files = glob.glob(h5_dir + '/**/*.h5', recursive=True)
len(h5_files) #should be 10,000

10000

In [3]:
#Reading the dataset
df = pd.read_csv("3593_song_newfeatures_genre_dataset.csv")

In [4]:
#Filtering out data for Electronic genre 
df_song = df[df['genre'] == 'Electronic']
df_song

,Unnamed: 0,id,spectral_centroid_mean,spectral_centroid_dev,rolloff_point_mean,rolloff_point_dev,flux_mean,flux_dev,variability_mean,variability_dev,compactness_mean,compactness_dev,rms_mean,rms_dev,low_energy_window_mean,low_energy_window_dev,zero_crossing_mean,zero_crossing_dev,title,genre
7,7,TRBGWJB128F4275C7B,9.461,14.330,0.05849,0.09428,0.003829,0.006677,0.004598,0.003710,1580.0,290.1,0.1706,0.12490,0.5809,0.06779,25.560,30.140,b'Psalm 27',Electronic
27,27,TRBGBDH12903CE9EA8,3.346,6.408,0.02144,0.04840,0.001876,0.002909,0.005946,0.002871,1582.0,196.3,0.1971,0.07106,0.5806,0.05834,9.810,16.350,b'Big Trouble Upstairs',Electronic
78,78,TRBGSCR128F92FE2AE,12.710,11.940,0.09153,0.09712,0.001476,0.002689,0.003095,0.001940,1576.0,193.5,0.1176,0.06459,0.6380,0.09078,40.440,28.330,b'Keep Hope Alive',Electronic
88,88,TRBGXBT128EF34EB68,6.566,3.830,0.03934,0.03174,0.005986,0.010130,0.006787,0.003362,1654.0,215.3,0.2493,0.10190,0.5527,0.05855,20.960,10.340,b'Momentum',Electronic
127,127,TRBHFZP12903CE88C3,9.823,14.640,0.05849,0.09144,0.001276,0.002220,0.002595,0.002417,1584.0,236.9,0.0916,0.08036,0.6272,0.04656,25.530,30.910,b'Fuck You',Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,TRAYHYA12903D15D6A,7.195,9.838,0.04726,0.07536,0.002558,0.004969,0.004755,0.003972,1376.0,442.8,0.1686,0.12160,0.5132,0.10970,21.150,24.580,b'Tokyo Freeze',Electronic
3497,3497,TRAYCUH128F9303485,4.483,5.194,0.02767,0.04904,0.003107,0.004934,0.005005,0.002810,1604.0,225.9,0.1751,0.08207,0.5391,0.05196,17.650,16.700,b'Yellow Moon',Electronic
3563,3563,TRAYSPP12903CEB12F,2.436,2.377,0.01661,0.01804,0.002487,0.002889,0.004540,0.001856,1593.0,194.0,0.1598,0.05054,0.5542,0.04691,7.524,6.594,b'Smpty spaces missing units',Electronic
3584,3584,TRAYKVH128F42AC993,16.600,11.970,0.12430,0.11210,0.004395,0.005453,0.005352,0.002298,1554.0,184.3,0.2147,0.08013,0.5737,0.04782,54.040,25.820,b'Mass Destruction',Electronic


In [5]:
#Creating lists of Genre and Track_id
track_id = df_song['id'].tolist()
genre = df_song['genre'].tolist()

In [6]:
genre

['Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electronic',
 'Electron

In [7]:
#Function to extract the cleaned track_id
def clean(text):
    text = str(text)
    text = text.replace("b'", "")
    text = text[:-1]
    return text

In [8]:
#Collecting segment info for Loudness only for those songs in the 3953_dataset
batch_size = 10

song_id = []
loud_max_seg = []
pitch_seg = []
timbre_seg = []

for i in range(0, len(h5_files), batch_size):
    # Get the batch of files
    batch_files = h5_files[i:i+batch_size]
    
    # Loop through the files in the batch
    for file_name in batch_files:
        h5 = hd.open_h5_file_read(file_name)
        
        id = hd.get_track_id(h5)
        id = clean(id)

        if id in track_id:
            #print("Collecting data")
            song_id.append(hd.get_track_id(h5))
            loud_max_seg.append(hd.get_segments_loudness_max(h5))
            timbre_seg.append(hd.get_segments_timbre(h5))
            pitch_seg.append(hd.get_segments_pitches(h5))
        
        h5.close()

In [9]:
#All are equal to the number of songs filtered out from the Electronic genre
len(song_id), len(loud_max_seg), len(timbre_seg), len(pitch_seg), len(genre)

(227, 227, 227, 227, 227)

In [10]:
#Inspecting values of loudness for different segments for 2 songs.
loud_max_seg[0:2]

[array([-60.   , -21.11 , -23.273, -23.948, -21.107, -23.262, -23.332,
        -14.969, -17.473, -15.537, -18.056, -18.102, -16.454, -17.335,
        -13.098, -15.398, -14.952, -18.001, -14.411, -16.902, -16.402,
        -16.845, -19.688, -19.969, -14.69 , -19.156, -16.052, -14.138,
        -17.874, -18.029, -18.097, -15.509, -16.817, -13.524, -14.734,
        -14.454, -14.813, -14.065, -17.196, -17.288, -16.702, -11.738,
        -12.462, -13.28 , -14.103, -13.548, -15.497, -17.485, -14.745,
        -13.424, -12.253, -12.744, -13.097, -13.438, -12.712, -15.562,
        -16.609, -15.223, -17.106, -18.772, -13.942, -16.276, -13.749,
        -15.68 , -14.082, -15.868, -17.533, -14.971, -13.744, -12.483,
        -13.164, -13.484, -13.779, -13.543, -15.294, -16.582, -15.222,
        -18.899, -14.831, -15.387, -16.308, -13.033, -15.272, -14.012,
        -15.695, -17.454, -14.967, -13.695, -12.352, -13.516, -13.715,
        -13.801, -13.494, -15.304, -16.509, -15.02 , -16.456, -18.497,
      

In [11]:
#inspecting the segments' shape for all songs
for arr in loud_max_seg:
    print(arr.shape)

(698,)
(885,)
(2085,)
(2890,)
(1235,)
(1509,)
(704,)
(134,)
(1797,)
(803,)
(2593,)
(729,)
(1131,)
(3391,)
(1200,)
(1347,)
(2463,)
(2391,)
(2117,)
(1894,)
(103,)
(1299,)
(1908,)
(1470,)
(357,)
(1487,)
(1202,)
(1245,)
(1793,)
(891,)
(2564,)
(1681,)
(1789,)
(1905,)
(987,)
(2400,)
(2460,)
(1151,)
(3992,)
(2026,)
(2813,)
(1036,)
(1544,)
(2094,)
(1736,)
(1009,)
(695,)
(1674,)
(2042,)
(1737,)
(1004,)
(245,)
(891,)
(703,)
(1364,)
(747,)
(1869,)
(680,)
(796,)
(354,)
(705,)
(3337,)
(770,)
(1873,)
(1482,)
(2088,)
(866,)
(954,)
(1945,)
(1606,)
(3170,)
(1513,)
(903,)
(1050,)
(1814,)
(1738,)
(828,)
(856,)
(1353,)
(580,)
(1468,)
(1026,)
(716,)
(744,)
(3841,)
(1194,)
(1258,)
(1067,)
(3106,)
(913,)
(1223,)
(1261,)
(1537,)
(88,)
(1298,)
(2786,)
(854,)
(825,)
(979,)
(1386,)
(2484,)
(805,)
(1796,)
(2511,)
(581,)
(2853,)
(992,)
(1245,)
(1029,)
(1323,)
(1712,)
(2344,)
(903,)
(2274,)
(1528,)
(2425,)
(1921,)
(1015,)
(3038,)
(1517,)
(842,)
(634,)
(1122,)
(1036,)
(429,)
(2388,)
(2849,)
(943,)
(879,)
(1045,)
(18

In [12]:
#Installing Librosa Library for creating spectrogram
!pip install librosa
import librosa
import librosa.display
import numpy as np

In [13]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Padding the loudness arrays to be of the same length 
max_length = max(len(arr) for arr in loud_max_seg)
padded_loud_max_seg = []
for arr in loud_max_seg:
    padded_arr = np.pad(arr, (0, max_length - len(arr)), mode='constant')
    padded_loud_max_seg.append(padded_arr)
concatenated_loudness = np.vstack(padded_loud_max_seg)

# Standardization (Z-score normalization)
mean_val = np.mean(concatenated_loudness)
std_val = np.std(concatenated_loudness)
scaled_loudness = (concatenated_loudness - mean_val) / std_val

sr= 22050

# Computing the spectrogram using librosa
spec = librosa.feature.melspectrogram(y=None, sr=sr, S=scaled_loudness, n_fft=2048, hop_length=512, n_mels=128)

# Converting the power spectrogram to dB scale
spec_db = librosa.power_to_db(spec, ref=np.max)

# Plotting the spectrogram
plt.figure(figsize=(10, 5))
librosa.display.specshow(spec_db, x_axis='time', y_axis='mel', sr=sr, hop_length=512, cmap='coolwarm')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram for Electronic genre')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()